In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Model name on Hugging Face
model_name = "sshleifer/distilbart-cnn-12-6"

# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\sayed\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sayed\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [6]:
text = """
    The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog.
    The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog.
    """

inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


 The quick brown fox jumps over the lazy dog . Quick brown fox jumpes over lazy dog in a bid to keep it on top of the dog . The foxes are often confused for each other and confused for the dog's identity . The quick foxes jump over lazy dogs in a series of different ways .


In [ ]:
from datasets import load_dataset
from rouge_score import rouge_scorer

# Step 3: Load the CNN/DailyMail dataset from Hugging Face
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Step 4: Define the ROUGE scorer
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Step 5: Function to generate summary and evaluate using ROUGE
def evaluate_summary(example):
    # Tokenize the input text (article)
    inputs = tokenizer(example["article"], return_tensors="pt", max_length=1024, truncation=True)
    
    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Get the reference summary (highlights) for ROUGE score calculation
    reference_summary = example["highlights"]
    scores = scorer.score(reference_summary, generated_summary)
    
    return scores

# Step 6: Evaluate on the first few examples in the dataset
results = []
for i in range(5):  # You can change 5 to a higher number to evaluate more examples
    example = dataset["test"][i]
    scores = evaluate_summary(example)
    results.append(scores)

# Step 7: Print ROUGE scores for evaluation
for i, scores in enumerate(results):
    print(f"Example {i + 1}:")
    print(f"ROUGE-1: {scores['rouge1']}")
    print(f"ROUGE-2: {scores['rouge2']}")
    print(f"ROUGE-L: {scores['rougeL']}")
    print("\n")